In [3]:
import time
import math
import hashlib

from collections import defaultdict

from IPython.display import display, HTML, clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))

dirs = {
    '>': (1, 0),
    '<': (-1, 0),
    '^': (0, -1),
    'v': (0, 1)
}

def get_next_blizzards(blizzards, max_x, max_y):
    new = []
    for d, coord in blizzards:
        x, y = coord
        dx, dy = dirs[d]
        nx, ny = (x+dx, y+dy)
        
        if nx == 0:
            nx = max_x - 1
        elif nx == max_x:
            nx = 1
        elif ny == 0:
            ny = max_y - 1
        elif ny == max_y:
            ny = 1
            
        new.append((d, (nx, ny)))
    return new
        
def get_next_possible_moves(start, blizzards, empty, max_x, max_y):
    moves = set(
        [
            (start[0]+dx, start[1]+dy) for dx, dy in dirs.values()
        ],
    )
    
    candidates = moves.intersection(empty)
    candidates.add(start)
    return candidates

def get_empty_spots(blizzards, max_x, max_y):
    coords = set(coord for d, coord in blizzards)
    empty = set([(1, 0), (max_x-1, max_y)])
    for x in range(1, max_x):
        for y in range(1, max_y):
            if (x, y) not in coords:
                empty.add((x, y))
                
    return empty

def visualize(loc, blizzards, x1, y1, x2, y2):
    
    coord_to_blizzards = defaultdict(list)
    for d, coord in blizzards:
        coord_to_blizzards[coord].append((d, coord))

    start,end = (1, 0), (x2-1, y2)
    rows = []
    for y in range(y1, y2+1):
        row = ''
        for x in range(x1, x2+1):
            coord = (x, y)
            if (x,y) == start or (x,y) == end:
                row += '.'
            elif x in [x1, x2] or y in [y1, y2]:
                row += '#'
            elif coord in coord_to_blizzards:
                if loc == coord:
                    row += 'X'
                    continue
                blizzards_at_coord = coord_to_blizzards[coord]
                
                if len(blizzards_at_coord) == 1:
                    row += blizzards_at_coord[0][0]
                else:
                    row += str(len(blizzards_at_coord))
            elif coord == loc:
                row += 'E'
            else:
                row += '.'
                
        rows.append(row)
    return '\n'.join(rows)

def get_key(blizzards):
    coord_to_blizzards = defaultdict(list)
    for d, coord in blizzards:
        coord_to_blizzards[coord].append((d, coord))

    key = ''
    for coord in sorted(coord_to_blizzards.keys()):
        blizzards = coord_to_blizzards[coord]
        key += f"{coord}-{len(blizzards)}" 
        
    result = hashlib.md5(key.encode())

    return result.hexdigest()

def get_mod_step_to_state(initial_state, max_x, max_y):
    steps = 0
    cache = []
    keys = set()
    blizzards = initial_state
    while True:
        key = get_key(blizzards)
        if steps % 1000 == 0:
            print('steps', steps, key)
        
        if key in keys:
            print('cycle', steps, max_x-1, max_y-1)
            break
        keys.add(key)
        cache.append((key, blizzards, get_empty_spots(blizzards, max_x, max_y)))
        
        blizzards = get_next_blizzards(blizzards, max_x, max_y)
        steps += 1
    return cache

def make_journey(start, end, blizzards, max_x, max_y):
    mod_step_to_state = get_mod_step_to_state(blizzards, max_x, max_y) 
    print(len(mod_step_to_state))
    q = [(0, start, blizzards)]
    seen = set([(0, get_key(blizzards))])
    while q:
        steps, loc, blizzards = q.pop(0)
        if steps % len(mod_step_to_state) == 0:
            print(steps, loc)
        s_b = set([b for d, b in blizzards])
        if loc in s_b:
            continue
        
        key, next_blizzards, empty = mod_step_to_state[steps % len(mod_step_to_state)]

        next_possible_moves = get_next_possible_moves(loc, next_blizzards, empty, max_x, max_y)
        
        for move in next_possible_moves:

            if move == end:
                print('done', steps, move)
                return steps, next_blizzards
            state = (move, key)
            if state in seen:
                continue
            q.append((steps+1, move, next_blizzards))
            seen.add(state)

def run():
    with open('../inputs/24.txt') as f:
        y = 0
        blizzards = []
        start = (1, 0)
        max_x, max_y = 0, 0
        for line in f:
            line = line.strip()
            for x, ch in enumerate(line):
                if ch in dirs.keys():
                    blizzards.append((ch, (x, y)))
                max_x, max_y = max(x, max_x), max(y, max_y)
            y+=1
        end = (max_x-1, max_y)
        
    s1, last_blizzards = make_journey(start, end, blizzards, max_x, max_y)
    print(s1)
    s2, last_blizzards = make_journey(end, start, last_blizzards, max_x, max_y)
    print(s2)
    s3, last_blizzards = make_journey(start, end, last_blizzards, max_x, max_y)
    
    print('answer', s1 + s2 + s3)
        
run()

steps 0 f4324889dc5df82b4fa72a97b59a949b
cycle 600 120 25
600
0 (1, 0)
done 288 (120, 26)
288
steps 0 803ebdbec0cf334ee13e6943a9eb55ee
cycle 600 120 25
600
0 (120, 26)
done 283 (1, 0)
283
steps 0 3c79cd6d6cee5e4a826d6557b9e2564f
cycle 600 120 25
600
0 (1, 0)
done 290 (120, 26)
answer 861
